In [1]:
import pandas as pd
# import os
# import lxml
# import DB as db

# test pandas 2.0

In [ ]:
# fn = r"C:\Users\yahia\OneDrive - Data and Transaction Services\Python-data\PortalLogs\summary\log summary-2023-04-03.zip"
# fn = r"C:\Users\yahia\OneDrive - Data and Transaction Services\Python-data\PortalLogs\logs\server.log.2023-02-23.zip"
fn= r"C:\Users\yahia\OneDrive - Data and Transaction Services\Python-data\PortalLogs\summary\log summary-2023-02-23-YJ-DELL.zip"
# %timeit df= pd.read_csv(fn)
df= pd.read_csv(fn, dtype_backend = 'pyarrow', low_memory=False)
# pd.options.mode.dtype_backend = 'pyarrow'


df.info()

In [ ]:
proj_fn = r"C:\Users\yahia\OneDrive - Data and Transaction Services\Projects\Realestate Reservation Portal\5- Operation\Incident reports\Reservation with status N\Copy of mpc_pro.xls"
pro_2_fn = r"C:\Users\yahia\OneDrive - Data and Transaction Services\Projects\Realestate Reservation Portal\5- Operation\Incident reports\Reservation with status N\Copy of soc_pro.xls"
data_types = {'الرقم القومى للزوج / ': 'string[pyarrow]', 
              'الرقم الأرضى': 'string[pyarrow]',
             'الرقم المحمول': 'string[pyarrow]',
             'الرقم القومى': 'string[pyarrow]',
             'رقم الأستمارة': 'string[pyarrow]'}
rdf_1= pd.read_excel(proj_fn, dtype_backend = 'pyarrow', skiprows=2 , dtype=data_types)
rdf_2= pd.read_excel(pro_2_fn, dtype_backend = 'pyarrow', skiprows=0 , dtype=data_types)
rdf =  pd.concat([rdf_1, rdf_2])
rdf.info()

In [ ]:
ip = df.query("token == 'confirmLandReservation True'")[['NID', 'IP_address', 'country', 'log_date']]
ip.info()

In [ ]:
df.query("token == 'confirmLandReservation True' and NID == '25006192300285'")

In [ ]:
rdf.merge(ip, how='left',right_on= 'NID', left_on=rdf.columns[15]).to_excel('./out/reservatio_with_N.xlsx')

In [ ]:
lst=[1,2,3,4,5,6,7]
li = [100,122,233,222,333,444, 555]

pd.DataFrame(lst, index=li)

# test

In [65]:
fn = r"C:\Users\yahia\OneDrive - Data and Transaction Services\Documents\Project 84 and 85.csv"
df = pd.read_csv(fn, low_memory=False, parse_dates=['status_date']).dropna(subset=['land_id'], axis=0)
df['res_hour'] = df.status_date.dt.hour

In [66]:
df.columns

Index(['land_application_id', 'payment_method_id', 'payment_receipt_id',
       'payment_bank_code', 'payment_amount', 'payment_percentage',
       'payment_date', 'land_application_date', 'applicant_national_id',
       'applicant_printed_image',
       ...
       'project_reservation_number', 'profile_data_exits',
       'profile_application_date', 'land_app_bank_id', 'land_app_bank_branch',
       'check_sum', 'check_sum_B', 'selected_governorate_id',
       'selected_city_id', 'res_hour'],
      dtype='object', length=102)

In [76]:
df1 =df1[['status_date', 'project_id', 'res_hour']].pivot_table(columns='project_id', index='res_hour', aggfunc = 'count').reset_index()
df1#.drop(columns = 'project_id' , axis=1, )#.plot()
# df1.index

KeyError: "['project_id'] not in index"

In [36]:
df[['status_date', 'project_id', 'res_hour']]

,status_date,project_id,res_hour
0,2023-01-15 00:01:59.000,84,0
1,2023-01-15 00:35:27.000,84,0
2,2023-01-15 00:56:18.000,84,0
3,2023-03-17 21:58:13.578,84,21
4,2023-01-15 01:05:15.000,84,1
...,...,...,...
8625,2023-02-20 21:51:15.000,85,21
8626,2023-02-23 10:00:31.000,85,10
8627,2023-02-23 10:01:45.000,85,10
8628,2023-02-23 10:00:32.000,85,10
